In [1]:
import json 
import ijson
import pandas as pd
import re, os
import random
from collections import Counter

In [2]:
country_code_df= pd.read_csv('database/country_code.csv' ,sep = ',')
country_code = country_code_df.values.tolist()

domain_df= pd.read_csv('database/domain.csv' ,sep = '.')
domain_code = domain_df.values.tolist()

flags_df= pd.read_csv('database/flags_new.csv' ,sep = ',')
flags = flags_df.values.tolist()

language_code_df= pd.read_csv('database/language_code.csv' ,sep = ':')
language_code = language_code_df.values.tolist()
    
cities_df= pd.read_csv('database/worldcities.csv' ,sep = ',')
cities = cities_df.values.tolist()

country_to_code = [row[1] for row in country_code]
country_class = { name: k+1 for k,name in enumerate(country_to_code) }#create class for each country code


In [3]:
### all the functiones needed for this project
def tokenize(s): return re.sub('([^A-Za-z0-9 ]+)', ' \\1 ', s).split()  # add space around anything not alphanum
def sanify(s): return s.replace(':',';').replace('|','/')

def get_key(dicti,val):
    for key, value in dicti.items():
         if val == value:
             return key
 
    return "There is no such Key"

def writeToVWFile(filename, examples):
    with open(filename, 'w') as h:
        for ex in examples:
            h.write(ex.strip()+'\n')
def pop_2(tokens,n=2):# 
    pop_list =[]
    for ii in tokens:
        if len(ii)<=n:
            pop_list.append(ii)
    for ii in pop_list:
        tokens.remove(ii)
    return tokens
    
def check_flag(tokens):
    labeled = False
    label = None
    #tokens = pop_2(tokens)
    for token in tokens: #check for countries with one part
        if token in flags_df.values:
            index_row = [flags.index(row) for row in flags if token in row[0]]
            if index_row:
                label_char =flags[index_row[0]][1]
                label_char =str(label_char).lower()
                label = country_class[label_char]
                labeled =True
                break
    return labeled , label

def city_country_check(place):
    is_labeled = False
    label = None
    place = pop_2(place,3)
    for token in place: #check for countries with one part
        if token in country_code_df.values :
            index_row = [country_code.index(row) for row in country_code if token in row[0]]
            label_char =country_code[index_row[0]][1]
            label_char =str(label_char).lower()
            if label_char in country_code_df.values :
                label = country_class[label_char]
                is_labeled= True
                break
    if not is_labeled:
        for k in range(len(place)-1): #for cities like new york or mexico city
            token ='%s %s' % (place[k],place[k+1])
            token = str(token).lower()
            #print(token)
            if token in cities_df.values :
                index_row = [cities.index(row) for row in cities if token == row[0] or token == row[4]]
                if index_row:
                    label_char =cities[index_row[0]][5]
                    label_char =str(label_char).lower()
                    if label_char in country_code_df.values :
                        label = country_class[label_char]
                        is_labeled= True
                        break
    if not is_labeled:
        for token in place: #check for cities with one part
            if token in cities_df.values :
                index_row = [cities.index(row) for row in cities if token == row[0] or token == row[4]]
                if index_row:
                    label_char =cities[index_row[0]][5]
                    label_char =str(label_char).lower()
                    if label_char in country_code_df.values :
                        label = country_class[label_char]
                        is_labeled= True
                        break
    return is_labeled , label

def operation(data):
    examples = []
    labels_list = []
    for i in range(len(data)):
        label = None
        namespaces = {}
        is_labeled= False
        if data[i]["place"] != None:
            #print(data[i]["place"])
            label_char = data[i]["place"]['country_code']
            label_char =str(label_char).lower()
            namespaces['p'] = [label_char]
            label = country_class[label_char]
            is_labeled= True

        if data[i]["user"]["location"] != "":
            place= tokenize(data[i]["user"]["location"].strip())
            place=pop_2(place)
            text = []
            text += place
            #print(place)
            namespaces['l'] = text
            if not is_labeled:
                is_labeled , label =city_country_check(place)

        if data[i]["user"]["name"] != "" and not is_labeled: only namespac
            user_name= tokenize(data[i]["user"]["name"].strip())
            namespaces['n'] = user_name
            if not is_labeled:
                is_labeled , label = check_flag(user_name)
            if not is_labeled:
                is_labeled , label =city_country_check(user_name)

        if data[i]["user"]["screen_name"] != "" and not is_labeled: only namespac
            screen_name= tokenize(data[i]["user"]["screen_name"].strip())
            namespaces['s'] = screen_name
            if not is_labeled:
                is_labeled , label = check_flag(screen_name)
            if not is_labeled:
                is_labeled , label =city_country_check(screen_name)

        if data[i]["user"]["description"] != "" and not is_labeled:
            description= tokenize(data[i]["user"]["description"].strip())
            namespaces['d'] = description
            if not is_labeled:
                is_labeled , label = check_flag(description)
            if not is_labeled:
                is_labeled , label =city_country_check(description)

        if data[i]["full_text"] != "":
            full_text= tokenize(data[i]["full_text"].strip())
            full_text = pop_2(full_text , 1)
            namespaces['f'] = full_text
            if not is_labeled:
                is_labeled , label = check_flag(full_text)

        if not is_labeled:# only namespac
            if 'url' in data[i]["user"]["entities"]:
                if 'display_url' in data[i]["user"]["entities"]['url']['urls'][0]:
                    if data[i]["user"]["entities"]['url']['urls'][0]['display_url'] != 'None':
                        url = data[i]["user"]["entities"]['url']['urls'][0]['display_url']
                        #print(url)
                        x = re.search("(\.[^c][^o]($|\W))", url)
                        #print(x)
                        if x!=None:
                            x = re.search("(\w\w)", x[0])
                            if x!=None:
                                if x[0] in country_code_df.values:
                                    namespaces['m'] = x[0] 
        if data[i]["lang"] != "":# only namespac
            lang= tokenize(data[i]["lang"].strip())
            #print(lang)
            if lang[0] in language_code_df.values:
                index_row = [language_code.index(row) for row in language_code if lang[0] in row]
                #print(language_code[index_row[0]][0])
                namespaces['g'] = [language_code[index_row[0]][0]]
                
        if label!=None:
            labels_list.append(label)
            ex = ''
            for ns,words in namespaces.items():
                ex += ' |%s %s ' % (ns, ' '.join(map(sanify,words)))
            examples += [ex]
    return examples , labels_list

def write_files(file_name,examples):
    random.seed(9999)
    random.shuffle(examples)
    middle = int(len(examples)/2)
    train_data = examples[:middle]
    test_data = examples[middle:]
    test_file_name = file_name+".te"
    train_file_name = file_name+".tr"
    with open(test_file_name, 'w') as h:
        for ex in test_data:
                h.write(ex.strip()+'\n')
    with open(train_file_name, 'w') as m:
        for ex in train_data:
                m.write(ex.strip()+'\n')
    return

def take_top_n(labels_list,n):# find top n-1 + others
    res = Counter(labels_list)
    items = list(res.items())
    sorted_list = sorted(items,key=lambda x: (x[1],x[0]),reverse=True)
    fir = sorted_list[:n-1]
    rest = sorted_list[n-1:]
    contry_code_list = [i[0] for i in fir]
    new_country_class = { name: k+1 for k,name in enumerate(contry_code_list) }
    return contry_code_list,new_country_class

def create_new_label(labels_list,n):# to reduce the number of classe to n
    contry_top_list,top_country_class=take_top_n(labels_list,n)
    new_label_list =[]
    for i in range(len(labels_list)):
        label = labels_list[i]
        if label in contry_top_list:
            new_label = top_country_class[label]
        else: 
            new_label = n
        new_label_list.append(new_label)
    return new_label_list, contry_top_list

def add_labels(examples , labels_list):
    new_examples = []
    for i in range(len(labels_list)):
        ex = str(labels_list[i])
        ex +='%s' % examples[i]
        new_examples += [ex]
    return new_examples

def create_vw_file(json_file,n = 20):# write the file name without .json
    with open(json_file+".json","r") as f:
        data=json.load(f)

    examples1 , labels_list1 = operation(data)
    print(labels_list1, examples1)
    new_label_list , contry_top_list= create_new_label(labels_list1,n)
    #top_country_code_list = [get_key(country_class,val-1) for val in new_label_list]
    #print(top_country_code_list)
    #top_country_list=[]
    #for token in top_country_code_list:
    #        top_country_list.append([row[0] for row in country_code if token in row[1]])
    new_examples = add_labels(examples1 , new_label_list)
    write_files("project/%s" %json_file,new_examples)
    return contry_top_list

In [4]:
top_country_list = create_vw_file("ramadan_l")

[157, 195, 239, 236, 195, 112, 236, 226, 195, 195, 195, 103, 226, 1, 104, 195, 162, 195, 195, 228, 195, 195, 195, 239, 168, 127, 60, 46, 4, 195, 157, 195, 195, 110, 136, 239, 199, 157, 15, 195, 238, 238, 195, 195, 236, 124, 238, 239, 195, 195, 195, 195, 239, 195, 111, 167, 151, 11, 195, 178, 195, 195, 168, 151, 239, 175, 195, 195, 195, 166, 239, 238, 195, 195, 162, 195, 103, 195, 135, 195, 206, 239, 120, 236, 195, 162, 151, 227, 175, 195, 209, 195, 166, 195, 195, 195, 195, 195, 120, 195, 195, 127, 4, 211, 112, 195, 195, 195, 195, 195, 195, 195, 32, 195, 166, 195, 4, 66, 195, 195, 195, 195, 195, 162, 66, 195, 195, 211, 195, 162, 195, 239, 162, 162, 209, 195, 168, 195, 211, 228, 238, 195, 195, 195, 239, 22, 195, 195, 239, 195, 195, 195, 18, 195, 195, 22, 4, 195, 4, 239, 195, 195, 238, 206, 104, 195, 162, 162, 195, 238, 195, 76, 195, 195, 195, 66, 19, 116, 239, 15, 195, 195, 239, 195, 234, 124, 57, 239, 195, 195, 195, 239, 239, 195, 195, 239, 110, 195, 166, 195, 239, 195, 195, 195, 195, 2

In [6]:
print(top_country_list)
print([country_code[i-1][0] for i in top_country_list])

[195, 239, 168, 120, 66, 236, 104, 4, 103, 238, 167, 162, 76, 157, 250, 228, 166, 18, 211]
['saudi arabia', 'united states', 'pakistan', 'kuwait', 'egypt', 'emirates', 'indonesia', 'algeria', 'india', 'united kingdom', 'oman', 'nigeria', 'france', 'netherlands', 'yemen', 'turkey', 'norway', 'bahrain', 'sudan']


In [44]:
!vw -k -c -b 27 --oaa 20 -d project/ramadan_l.tr -f project/ramadan_l.model --passes 20 --holdout_after 650 -q pp -q pl -qpg -q lg

final_regressor = project/ramadan_l.model
Num weight bits = 27
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = project/ramadan_l.tr.cache
Reading datafile = project/ramadan_l.tr
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0       19        1       24
1.000000 1.000000            2            2.0        1       19       17
0.750000 0.500000            4            4.0        1        1       54
0.625000 0.500000            8            8.0        1        1       15
0.625000 0.625000           16           16.0        2        1       28
0.562500 0.500000           32           32.0        1        1       33
0.593750 0.625000           64           64.0        9        1       36
0.539062 0.484375          128          128.0        1        1       22
0.503906 0.468750          256

In [43]:
!vw -t -i project/ramadan_l.model -d project/ramadan_l.te

creating quadratic features for pairs: pp pl pg lg 
only testing
Num weight bits = 27
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = project/ramadan_l.te
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1       37
0.000000 0.000000            2            2.0       13       13       63
0.000000 0.000000            4            4.0        1        1        9
0.125000 0.250000            8            8.0        4        4       52
0.125000 0.125000           16           16.0        2        2       45
0.250000 0.375000           32           32.0        1        1       21
0.218750 0.187500           64           64.0        1        1       20
0.187500 0.156250          128          128.0       10       20       18
0.246094 0.304688          256          256.0        6        6  

# Next
-b 24 --affix +6, -3 --spelling _ --ngram 2 --loss_function logistic --loss_function hinge -q wl -q ll -q ww --nn 10 --inpass --dropout


In [25]:
!vw -k -c -b 27  --oaa 20 -d project/ramadan_l.tr -f project/ramadan_l2.model --passes 20 --holdout_after 600 --affix +5,-3 #-q pl --nn 8 --inpass --dropout #-q ww

final_regressor = project/ramadan_l2.model
Num weight bits = 27
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = project/ramadan_l.tr.cache
Reading datafile = project/ramadan_l.tr
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0       19        1       24
1.000000 1.000000            2            2.0        1       19       17
0.750000 0.500000            4            4.0        1        1       54
0.625000 0.500000            8            8.0        1        1       15
0.625000 0.625000           16           16.0        2        1       28
0.562500 0.500000           32           32.0        1        1       33
0.593750 0.625000           64           64.0        9        1       36
0.539062 0.484375          128          128.0        1        1       22
0.503906 0.468750          25

In [18]:
!vw -t -i project/ramadan_l2.model -d project/ramadan_l.te

creating quadratic features for pairs: pl 
only testing
using dropout for neural network testing
using input passthrough for neural network testing
Num weight bits = 27
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = project/ramadan_l.te
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1       33
0.500000 1.000000            2            2.0       13        3       61
0.250000 0.000000            4            4.0        1        1        9
0.250000 0.250000            8            8.0        4        4       49
0.250000 0.250000           16           16.0        2        2       43
0.406250 0.562500           32           32.0        1        1       19
0.359375 0.312500           64           64.0        1       20       18
0.296875 0.234375          128          128.0       10 